In [1]:
# ===================================================================
# Proyek Portofolio #3: Prediksi Arah Harga Saham dengan Machine Learning
# Tujuan: Membangun dan mengevaluasi model untuk memprediksi
#         apakah harga saham akan NAIK atau TURUN pada hari berikutnya.
# ===================================================================

# --- Bagian 1: Konfigurasi (Ubah parameter di sini untuk eksperimen) ---
ticker_saham = 'TLKM.JK'  # Ganti dengan ticker saham syariah pilihan Anda
tanggal_mulai = '2015-01-01'
tanggal_akhir = '2025-09-15' # Bisa diganti dengan tanggal hari ini

# Parameter Indikator untuk Fitur
sma_cepat_period = 50
sma_lambat_period = 200
rsi_period = 14
return_lag_period = 5

# Parameter Model
persen_data_latihan = 0.8
jumlah_pohon_model = 100 # n_estimators untuk Random Forest

# --- Bagian 2: Import Library ---
# Mengimpor semua "perkakas" yang kita butuhkan
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- Bagian 3: Akuisisi & Persiapan Data ---
print(f"--- Memulai Analisis untuk {ticker_saham} ---")
print("1. Mengunduh data historis...")

# Mengunduh data dari Yahoo Finance
df = yf.download(ticker_saham, start=tanggal_mulai, end=tanggal_akhir)

# Memeriksa apakah unduhan berhasil
if df.empty:
    print(f"GAGAL mengunduh data untuk {ticker_saham}. Proses dihentikan.")
else:
    # Meratakan nama kolom jika yfinance menggunakan MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    print("   Data berhasil diunduh.")

    # --- Bagian 4: Feature Engineering (Membuat "Petunjuk" untuk Model) ---
    print("2. Membuat Fitur (Features) dan Target...")
    
    # Menghitung Indikator Teknis
    df['SMA_Cepat'] = df['Close'].rolling(window=sma_cepat_period).mean()
    df['SMA_Lambat'] = df['Close'].rolling(window=sma_lambat_period).mean()
    # Menggunakan ta.rsi() secara langsung dan spesifik pada kolom 'Close'
    df['RSI'] = ta.rsi(close=df['Close'], length=rsi_period)
    
    # Membuat Fitur dari Indikator
    df['Fitur_Jarak_SMA'] = df['Close'] - df['SMA_Cepat']
    df['Fitur_RSI'] = df['RSI']
    df['Fitur_Return'] = df['Close'].pct_change(periods=return_lag_period)
    
    # Membuat Target (Jawaban yang ingin ditebak)
    # Target = 1 jika harga besok NAIK, 0 jika sebaliknya
    df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
    print("   Fitur dan Target berhasil dibuat.")

    # --- Bagian 5: Persiapan Data untuk Model ---
    print("3. Mempersiapkan data (membersihkan dan membagi)...")
    
    # Membuang semua baris yang memiliki nilai NaN (kosong)
    df_bersih = df.dropna()
    
    # Mendefinisikan kolom mana saja yang akan menjadi Fitur
    nama_fitur = ['Fitur_Jarak_SMA', 'Fitur_RSI', 'Fitur_Return']
    
    # Memisahkan antara Fitur (X) dan Target (y)
    X = df_bersih[nama_fitur]
    y = df_bersih['Target']
    
    # Membagi data menjadi data Latihan (Training) dan data Ujian (Testing)
    split = int(len(df_bersih) * persen_data_latihan)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    print(f"   Data Latihan: {len(X_train)} baris, Data Ujian: {len(X_test)} baris.")

    # --- Bagian 6: Pelatihan & Evaluasi Model ---
    print("4. Melatih model Machine Learning (Random Forest)...")
    
    # Membuat instance model (seperti membuat "otak" yang masih kosong)
    model = RandomForestClassifier(n_estimators=jumlah_pohon_model, random_state=42)
    
    # Melatih model menggunakan data latihan
    model.fit(X_train, y_train)
    print("   Model selesai dilatih.")

    print("5. Mengevaluasi performa model pada data ujian...")
    # Meminta model yang sudah dilatih untuk menebak jawaban dari data ujian
    prediksi = model.predict(X_test)
    
    # Menghitung metrik performa
    akurasi = accuracy_score(y_test, prediksi)
    laporan_klasifikasi = classification_report(y_test, prediksi)
    matriks_kebingungan = confusion_matrix(y_test, prediksi)

    # --- Bagian 7: Menampilkan Laporan Akhir ---
    print("\n" + "="*50)
    print(f"             HASIL ANALISIS MODEL UNTUK: {ticker_saham}")
    print("="*50)
    print(f"Periode Analisis: {tanggal_mulai} hingga {tanggal_akhir}")
    print(f"\nAkurasi Model: {akurasi:.2%}\n")
    
    print("--- Matriks Kebingungan (Confusion Matrix) ---")
    print("          Prediksi Turun | Prediksi Naik")
    print(f"Aktual Turun:      {matriks_kebingungan[0][0]}      |      {matriks_kebingungan[0][1]}")
    print(f"Aktual Naik :      {matriks_kebingungan[1][0]}      |      {matriks_kebingungan[1][1]}")
    
    print("\n--- Laporan Klasifikasi Rinci ---")
    print(laporan_klasifikasi)
    print("="*50)


--- Memulai Analisis untuk TLKM.JK ---
1. Mengunduh data historis...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


   Data berhasil diunduh.
2. Membuat Fitur (Features) dan Target...
   Fitur dan Target berhasil dibuat.
3. Mempersiapkan data (membersihkan dan membagi)...
   Data Latihan: 1948 baris, Data Ujian: 488 baris.
4. Melatih model Machine Learning (Random Forest)...
   Model selesai dilatih.
5. Mengevaluasi performa model pada data ujian...

             HASIL ANALISIS MODEL UNTUK: TLKM.JK
Periode Analisis: 2015-01-01 hingga 2025-09-15

Akurasi Model: 50.41%

--- Matriks Kebingungan (Confusion Matrix) ---
          Prediksi Turun | Prediksi Naik
Aktual Turun:      161      |      102
Aktual Naik :      140      |      85

--- Laporan Klasifikasi Rinci ---
              precision    recall  f1-score   support

           0       0.53      0.61      0.57       263
           1       0.45      0.38      0.41       225

    accuracy                           0.50       488
   macro avg       0.49      0.49      0.49       488
weighted avg       0.50      0.50      0.50       488



## Prediksi Arah Harga Saham Syariah dengan Machine Learning

### **1. Ringkasan Eksekutif**
Proyek ini bertujuan untuk membangun dan mengevaluasi sebuah model machine learning (Random Forest) yang dapat memprediksi arah pergerakan harga harian (Naik/Turun) untuk saham syariah di Bursa Efek Indonesia. Dengan menggunakan data historis lebih dari 10 tahun, kami menciptakan serangkaian fitur berbasis indikator teknikal seperti SMA dan RSI. Hasil evaluasi model pada data yang belum pernah dilihat sebelumnya menunjukkan Akurasi 50.41% yang mengindikasikan adanya potensi keunggulan statistik yang kecil namun perlu dioptimalkan lebih lanjut.

### **2. Metodologi**
Alur kerja proyek ini mengikuti standar praktik dalam analisis kuantitatif, yang terdiri dari beberapa tahap:

**a. Akuisisi & Persiapan Data**
 * **Sumber Data:** Data harga historis harian diunduh secara otomatis dari Yahoo Finance menggunakan library yfinance.
 * **Aset:** Analisis difokuskan pada saham TLKM.JK.
 * **Periode:** Data yang diambil mencakup periode dari 2025-01-01 hingga 2025-09-15.
 * **Data Utama:** Analisis menggunakan kolom harga penutupan yang disesuaikan (Adj Close) untuk akurasi perhitungan imbal hasil.

**b. Feature Engineering (Pembuatan Fitur)**
Untuk memberikan "petunjuk" kepada model, serangkaian fitur teknikal dibuat dari data harga mentah:
 * **Target:** Variabel target didefinisikan sebagai 1 jika harga pada hari berikutnya naik, dan 0 jika sebaliknya.
 * **Fitur Indikator:**
   * **Simple Moving Average (SMA):** Digunakan untuk mengidentifikasi tren jangka pendek dan menengah (SMA_50 dan SMA_200).
   * **Relative Strength Index (RSI):** Digunakan untuk mengukur momentum overbought/oversold (RSI_14).
   * **Return Historis:** Perubahan harga selama 5 hari terakhir digunakan sebagai indikator momentum jangka pendek.
   * **Jarak dari SMA:** Fitur tambahan dibuat untuk mengukur seberapa jauh harga saat ini dari trennya.

**c. Persiapan & Pelatihan Model**
* **Pembersihan Data:** Semua baris data yang mengandung nilai kosong (NaN)—yang muncul akibat perhitungan indikator—dihapus untuk memastikan kualitas data.
* **Pembagian Data:** Data yang bersih dibagi menjadi dua set: 80% data pertama untuk melatih model (Training Set) dan 20% data sisa untuk menguji performanya (Testing Set).
* **Model yang Digunakan:** Model Random Forest Classifier dipilih karena kemampuannya yang kuat dalam menangani hubungan non-linear yang kompleks dan ketahanannya terhadap overfitting.

### **3. Hasil & Evaluasi**
Model yang telah dilatih kemudian diuji pada Testing Set yang belum pernah ia lihat sebelumnya. Hasil performa dievaluasi menggunakan metrik standar industri.

**a. Akurasi Model**
Akurasi keseluruhan model pada data ujian adalah 50.41%. Ini menunjukkan bahwa model berhasil menebak arah pergerakan harga dengan benar dengan mendekati sekitar 51% dari waktu.

**b. Laporan Klasifikasi Rinci**
* **Presisi (Precision) Sinyal Naik (Label 1):** 45%
 * Artinya: Dari semua sinyal "Naik" yang diberikan model, 45% di antaranya benar.
**Cakupan (Recall) Sinyal Naik (Label 1):** 38%
 * Artinya: Dari semua hari di mana harga benar-benar naik, model berhasil menangkap 38% di antaranya.
 
### **4. Kesimpulan & Langkah Selanjutnya**

Meskipun model ini menunjukkan akurasi sedikit di atas 50%, yang mengindikasikan adanya edge statistik yang sangat tipis, performanya saat ini belum cukup kuat untuk diterapkan di pasar nyata. Rendahnya nilai precision dan recall menunjukkan bahwa model masih sering memberikan sinyal palsu dan melewatkan banyak peluang.

**Langkah selanjutnya untuk perbaikan adalah:**
* **1. Menambah Fitur:** Melakukan riset untuk menambahkan fitur-fitur yang lebih canggih (misalnya, volatilitas, data makroekonomi).
* **2. Optimalisasi Model:** Melakukan hyperparameter tuning untuk menemukan pengaturan terbaik dari model Random Forest.
* **3. Menguji Model Lain:** Membandingkan performa Random Forest dengan model lain seperti Gradient Boosting atau LSTM.

Proyek ini berhasil membangun sebuah alur kerja riset machine learning yang solid dan menjadi fondasi yang baik untuk eksperimen lebih lanjut.